# Computação Bioinspirada 2024-02

Este repositório contém o trabalho da Tópicos Avançados em Ciências de Computação II
(Computação Bioinspirada, no semestre 2024-02).

O trabalho consiste em trabalhar com um conjunto de dados multirrótulo onde cada instância representa uma sequência de
proteína. Cada rótulo (classe) corresponde a uma localização subcelular e as proteínas podem estar presentes
simultaneamente em dois ou mais compartimentos celulares. O conjunto de dados possui seis localizações
subcelulares: Proteínas do Capsídeo Viral, Proteínas da Membrana Celular do Hospedeiro, Proteínas do Retículo
Endoplasmático do Hospedeiro, Proteínas do Citoplasma do Hospedeiro, Proteínas do Núcleo do Hospedeiro e
Proteínas Secretadas. As colunas representam os códigos de Gene Ontology (relacionados à função da proteína),
com valores que indicam a frequência do código para cada proteína. As seis últimas colunas indicam a presença
(1) ou ausência (0) da proteína em cada uma das localizações subcelulares mencionadas.

Desenvolvemos um modelo de classificação multirrótulo usando **redes neurais artificiais**.

Existem dois conjuntos de dados: um de vírus e um de plantas.


### Imports 

In [335]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import random

In [314]:
random_seed = 42
tf.random.set_seed(random_seed)
tf.keras.utils.set_random_seed(random_seed)

### Lendo os dados dos datasets e separando em treino e teste

In [315]:
plants_dataset = pd.read_csv('./Plants_Dataset_Term_Frequency.tsv', sep='\t', skiprows=1).iloc[:, 1:]
virus_dataset =  pd.read_csv('./Virus_Dataset_Term_Frequency.tsv', sep='\t', skiprows=1).iloc[:, 1:]

In [316]:
# Separar as features e os rótulos para o dataset de plantas
X_plants = plants_dataset.iloc[:, :-6].values  # Todas as colunas, exceto as últimas 6 e a primeira
y_plants = plants_dataset.iloc[:, -6:].values  # As últimas 6 colunas (rótulos)

# Separar as features e os rótulos para o dataset de vírus
X_virus = virus_dataset.iloc[:, :-6].values
y_virus = virus_dataset.iloc[:, -6:].values

# Dividir dados em treino e teste
X_train_plants, X_test_plants, y_train_plants, y_test_plants = train_test_split(
    X_plants, y_plants, test_size=0.3, random_state=random_seed
)

X_train_virus, X_test_virus, y_train_virus, y_test_virus = train_test_split(
    X_virus, y_virus, test_size=0.3, random_state=random_seed
)

In [317]:

# Padronizar os dados
scaler_plants = StandardScaler()
X_train_plants = scaler_plants.fit_transform(X_train_plants)
X_test_plants = scaler_plants.transform(X_test_plants)

scaler_virus = StandardScaler()
X_train_virus = scaler_virus.fit_transform(X_train_virus)
X_test_virus = scaler_virus.transform(X_test_virus)

In [318]:
def hamming_loss(predictions, actual_values):
    N = len(predictions)
    L = 6

    s = 0

    for i in range(N):
        for j in range(L):
            s += predictions[i][j] ^ actual_values[i][j]
    
    return s / (N * L)

In [319]:
class NeuralNetwork():
    def __init__(self, in_sz, out_sz, h_layer_num, features):
        self.in_sz = in_sz
        self.out_sz = out_sz
        self.h_layer_num = h_layer_num
        self.layer_features = features
        self.model = None
        self.hamming_loss = 0

    def create_model(self):
        model = None
        model = Sequential()
        model.add(Input(shape=(self.in_sz,)))
        for neur_num, act_func in zip(self.layer_features[0], self.layer_features[1]):
            model.add(Dense(neur_num, activation=act_func))
        model.add(Dense(self.out_sz, activation='sigmoid'))  # Saída com sigmoid para classificação binária
        
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        self.model = model
    
    def train_model(self, X_train, y_train, epochs, batch, val_split):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch, 
                       validation_split=val_split, verbose=0)
    
    def model_loss(self, X_test, y_test):
        predictions_test = self.model.predict(X_test)
        predicted_test = (predictions_test > 0.5).astype(int)
        self.hamming_loss += hamming_loss(predicted_test, y_test)
        
        

In [320]:
# Criar e treinar o modelo para plantas
features = [[128, 64], ['relu', 'relu']]
nn = NeuralNetwork(X_train_plants.shape[1], y_train_plants.shape[1], 3, features)
nn.create_model()
history_plants = nn.train_model(X_train_plants, y_train_plants, epochs=50, batch=32, val_split=0.3)

In [321]:
# Criar e treinar o modelo para vírus
nn2 = NeuralNetwork(X_train_virus.shape[1], y_train_virus.shape[1], 3, features)
nn2.create_model()
history_virus = nn2.train_model(X_train_virus, y_train_virus, epochs=50, batch=32, val_split=0.3)

In [322]:
# Obter previsões para o conjunto de teste de plantas
predictions_plants_test = nn.model.predict(X_test_plants)

# Obter previsões para o conjunto de treino de plantas
predictions_plants_train = nn.model.predict(X_train_plants)

# Obter previsões para o conjunto de teste de vírus
predictions_virus_test = nn2.model.predict(X_test_virus)

# Obter previsões para o conjunto de treino de vírus
predictions_virus_train = nn2.model.predict(X_train_virus)


# Transformar as probabilidades em rótulos binários (0 ou 1) com um limiar de 0.5
predicted_classes_plants_test = (predictions_plants_test > 0.5).astype(int)
predicted_classes_plants_train = (predictions_plants_train > 0.5).astype(int)
predicted_classes_virus_test = (predictions_virus_test > 0.5).astype(int)
predicted_classes_virus_train = (predictions_virus_train > 0.5).astype(int)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [323]:
possibilidades = [
    [predicted_classes_plants_train, y_train_plants, 'dados de treino das plantas'],
    [predicted_classes_plants_test, y_test_plants, 'dados de teste das plantas'],
    [predicted_classes_virus_train, y_train_virus, 'dados de treino dos vírus'],
    [predicted_classes_virus_test, y_test_virus, 'dados de teste dos vírus']
]

In [324]:
for possibilidade in possibilidades:
    print(f'A hamming loss nos {possibilidade[-1]} foi de {100*hamming_loss(*possibilidade[:-1]):.2f}%')

A hamming loss nos dados de treino das plantas foi de 1.16%
A hamming loss nos dados de teste das plantas foi de 3.97%
A hamming loss nos dados de treino dos vírus foi de 3.15%
A hamming loss nos dados de teste dos vírus foi de 6.18%


In [ ]:
# possíveis valores para nossa população inicial

p_neur = [2**i for i in range(5, 10)]
p_h_layers = [i for i in range(2, 5)]
p_act_func = ['relu', 'gelu', 'leaky_relu', 'tanh']


In [ ]:
# gerando uma RN com parâmetros 'aleatórios' (dentro de opções pré definidas)

def gen_random_nn():
    layers = random.choice(p_h_layers)
    act_funcs = [random.choice(p_act_func) for _ in range(layers)]
    neur_num = [random.choice(p_neur) for _ in range(layers)]
    nn = NeuralNetwork(X_train_plants.shape[1], y_train_plants.shape[1], layers, [neur_num, act_funcs])
    return nn
        

In [ ]:
x = gen_random_nn()


0

## O que está feito -> 
Classe NeuralNetwork, que vai ser o indivíduo do nosso 
PSO

## O que precisa fazer? 
O PSO.

### Mais especificamente:

1. Gerar uma população de 12 indivíduos (existem motivos para esse número*) 
aleatórios
2. Inicializar a posição e velocidade
3. Realizar o treinamento de cada indivíduo e analisar a perda de hamming como fitness
4. Realizar a atualização da posição e velocidade de cada indivíduo
5. Voltar para 3 (posição e velocidade mudarão os parâmetros dos ind)

### Considerações sobre o algoritmo:

1. Quantas vizinhanças? Definir as vizinhanças por N de camdas? São poucos indivíduos
então seria computacionalmente barato iterar pela população
2. O split nos dados de treino e teste vai ser feito a cada geração? Um para cada
vizinhança, um para cada indivíduo?

## Quais serão nossos hiperparâmetros? 

Número de Neurônios por camada [32-256] passo de 16

Número de camadas escondidas [2-4]

Tipo da função de ativação [relu, gelu, leakyrelu, tanh]

## Considerações do que pode ser incluso como HPP

Número de Épocas

Tamanho do batch

validation split

??

